In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from statsmodels.tsa.api import Holt, SimpleExpSmoothing

In [2]:
sys.path.insert(1, '../')
from helpers.decision_maker import DecisionMaker
from seeds import known_seeds
from evaluation import get_actual_demand
from utils import save_solution
from utils import load_problem_data

In [3]:
def get_my_solution(d, p, seed):
    _, datacenters, servers, selling_prices, _ = load_problem_data(path='../data/')
    
    decision_maker = DecisionMaker(datacenters,servers,selling_prices, d, seed, p)

    fleet, prices, O = decision_maker.solve()

    return pd.DataFrame(fleet), pd.DataFrame(prices), O

In [4]:
def get_all_solutions(p, seed):
    demand = pd.read_csv('../data/demand.csv')

    # SET THE RANDOM SEED
    np.random.seed(seed)

    # GET THE DEMAND
    actual_demand = get_actual_demand(demand)

    # CALL YOUR APPROACH HERE
    _, _, O = get_my_solution(actual_demand, p, seed)
    
    return O

In [5]:
ps = [i/100 for i in range(15, 31, 1)]
seeds = [2521, 2381, 5351, 6047, 6829, 9221, 9859, 8053, 1097, 8677]



all_best_params = {}

for seed in seeds:
    best_params = {}

    params = []
    for i in range(len(ps)):
        
        p = random.choice(ps)
        while(p in params):
            p = random.choice(ps)
        params.append(p)
        score = get_all_solutions(p,seed)
        
        avg = score
        f = open('results.txt', 'a+')
        f.write("SEED#: " + str(seed) + " | score: " + str(avg) + " | lookahead: " +  str(p)  + "\n")
        f.close()
        if len(best_params)<5:
            best_params[avg] = p
        else:
            worst_score = sorted(best_params.keys())[0]
            if avg > worst_score:
                del best_params[worst_score]
                best_params[avg] = p

    print("SEED#: ", str(seed))
    print(best_params)
    all_best_params[seed] = best_params



1  OBJ: -7972537.0625
2  OBJ: -7686669.5
3  OBJ: -23444780.132469103
4  OBJ: -23419055.257469103
5  OBJ: -34326786.4707248
6  OBJ: -32530212.04816283
7  OBJ: -44560960.07769607
8  OBJ: -42408814.470469184
9  OBJ: -53905699.43367506
10  OBJ: -51057167.85343612
11  OBJ: -63719082.59857617
12  OBJ: -60269672.59129643
13  OBJ: -75450436.047993
14  OBJ: -71084652.91761017
15  OBJ: -82896458.12264839
16  OBJ: -77982541.03996779
17  OBJ: -88758575.45556647
18  OBJ: -82837393.03245315
19  OBJ: -90286399.07785068
20  OBJ: -83798744.87430555
21  OBJ: -87253375.51678944
22  OBJ: -80404296.0107082
23  OBJ: -73906070.40937316
24  OBJ: -66671908.4463826
25  OBJ: -66424012.20800239
26  OBJ: -58564307.16848087
27  OBJ: -53888939.996747226
28  OBJ: -45811295.82816337
29  OBJ: -37897179.230106555
30  OBJ: -29785229.656217676
31  OBJ: -21559632.406554095
32  OBJ: -13209827.03946418
33  OBJ: -9030685.322059315
34  OBJ: -607102.0409891084
35  OBJ: 7956152.51124784
36  OBJ: 16631350.738444533
37  OBJ: 42747